In [173]:
from modsim import *
import math
import pandas as pd
from random import randint 

# LIMESHARE 

- limeshare is...


**Below is our state with attributes:**
- number_of_scooters: 
- av_batt_life:
- size_of_region:
- send_truck:

In [163]:
limeshare = State(number_of_scooters = 20, av_batt_life = 100, size_of_region = 2, send_truck = -1)
limeshare

,values
number_of_scooters,20
av_batt_life,100
size_of_region,2
send_truck,-1


In [164]:
def step(state, hour):
#     find number of users per hour
    demand = calcDemand(state.number_of_scooters, hour)
    
#     find the average ride length per rider from demand
    length_of_use = calcAverageRide(state.size_of_region, demand)
    
#     calc change in battery 
    calcBattDelta(state, demand, length_of_use)
    
#     find hour to send truck
    if(state.av_batt_life < 20 and state.send_truck == -1):
        sendTruck(state, hour)


## Calculate demand for scooters based on: 
- traffic times 
    - HIGH : going to work (8-10pm), lunch time (12-2pm), going home (4-6pm)
    - NUETRAL : early birds (6-8am), night outs (8pm-11pm)
    - LOW : all remaining times
- scooters in the area (this is a hard max) 
- randomize using round (say bad weather hour)

*Returns num of scooters*

In [271]:
def calcDemand(scooters, hour):
    if hour >= 8 and hour < 10 or hour >= 12 and hour < 14 or hour >= 16 and hour < 18:
        return randint(int(scooters/hour) + int(hour/2), scooters)
    elif hour == 6 or hour == 7 or hour >= 20 and hour < 23:  
        return randint(scooters%(hour/2), scooters)
    else: 
        if hour == 0:
            return randint(hour, int(scooters/4))
        else:
            return randint(int(hour/6), int(scooters/4))

**The following code is used to test ideal hour ranges for our function above with demand.**
- the goal was to have a higher min for high traffic times accounting for hour and sccoter amounts range(20-100)


| trafffic | min  | max  | av |  
|:---|---|---|---|
| high  (13)| 13  |100   |   57.03|   
| nuetral  (20)| 1  | 100  |  48.34 |   
| low (4)|  0 |  25 |  12.55 |   
[This is for 100 scooters]


| trafffic | min  | max  | av |  
|:---|---|---|---|
| high  (13)|   7|20   |   13.08|   
| nuetral  (20)| 0  | 20  |  10.6 |   
| low (4)|  0 | 5  | 2.37  |   
[This is for 20 scooters]

In [277]:
hours, min, max, av = 4, 100, 0, 0
for i in range(1, 100):
    store = calcDemand(100, hours)
    if store < min: min = store
    if store > max: max = store
    av += store
print(min, max, av/100)

0 25 12.55


## Caluculate average ride length of scooters used 
- we are assuming that the user picks up a scooter — at worst — at the begining of the hour and returns it by the end of the hour 
- the average length per scooter ride thus is a range from 0 - 60 minutes 

**factors the affect the ride length**
- area of region (the higher the area the longer the ride)
- demand (if there is more demand, chances are the rides are longer)

*returns a number between 0-60*

In [286]:
def calcRide(area, demand):
    lower_bound = randint(0, demand%area)
    upper_bound = min(area*demand + lower_bound, 60)
    return randint(lower_bound, upper_bound)

def calcAverageRide(area, demand):
    av = 0
    for i in range (1, demand): 
        av += calcRide (area, demand);
    return av/demand

Test the method above with av demand of 20 and area range 2,3,7 (square miles)

In [287]:
calcAverageRide(2, 20)
calcAverageRide(3, 20)
calcAverageRide(7, 20)

TypeError: 'int' object is not callable

In [278]:
def calcBattDelta(state, demand, av_length_of_use):
    s = state.number_of_scooters 
    c = state.av_batt_life
    t = demand
    
    state.av_batt_life =  math.floor(((s-t)*c + t*(c- av_length_of_use)) / s)
        

def sendTruck(state, hour):
    state.send_truck = hour;

In [279]:
def run_simulation(state, num_steps):     
    results = TimeSeries()
    for i in range(num_steps):
        #  this is our line in the sand to prevent calculating after our desired purpose for the model has been found
        if(state.send_truck != -1):
            return
        step(state, i)
        results[i] = state.av_batt_life

In [280]:
limeshare = State(number_of_scooters = 20, av_batt_life = 100, size_of_region = 3, send_truck = -1)
run_simulation(limeshare, 24)
limeshare

ValueError: non-integer arg 1 for randrange()

In [ ]:
def runModel():
#     tuple array ( size_of_region, send_truck) with fixed number_of_scooters at 20
    regions = linspace(1, 5, 10)
    results = []
    for i in regions:
        limeshare = State(number_of_scooters = 20, av_batt_life = 100, size_of_region = round(i + 1, 2), send_truck = -1)
        run_simulation(limeshare, 24)
        data = (limeshare.size_of_region, limeshare.send_truck)
        results.append(data)
    return results 

In [ ]:
sample_regions = runModel()

In [ ]:
def runModel():
#     tuple array (number_of_scooters, send_truck) with fixed size_of_region at 2
    scooters = linspace(20, 70, 10)
    results = []
    for i in scooters:
        limeshare = State(number_of_scooters = int(i), av_batt_life = 100, size_of_region = 2, send_truck = -1)
        run_simulation(limeshare, 24)
        data = (limeshare.number_of_scooters, limeshare.send_truck)
        results.append(data)
    return results 

In [ ]:
sample_scooters = runModel()

In [ ]:
region_data = linspace(1, 5, 10)


In [ ]:
scooter_data = linspace(20, 70, 10)

In [ ]:
truck_data = [[0 for x in region_data] for y in scooter_data] 

In [ ]:
def runModel():
    for row, reigon in enumerate(region_data):
        for col, scooters in enumerate(scooter_data):
            limeshare = State(number_of_scooters = scooters, av_batt_life = 100, size_of_region = reigon, send_truck = -1)
            run_simulation(limeshare, 24)
            truck_data[row][col] = limeshare.send_truck

runModel()
truck_data

In [ ]:
# My reaction when the above code printed "HOLY SHIT!"

In [ ]:
new_dataFrame = pd.DataFrame(data=truck_data, index=round(region_data, 2), columns=round(scooter_data, 2))
new_dataFrame